# Compiling and Deploying Hugging Face Pre-trained BERT

### Introduction

In this tutorial we will compile and deploy the BERT-base version of Hugging Face 🤗 Transformers BERT for Inferentia. The full list of HuggingFace's pretrained BERT models can be found in the BERT section on this page https://huggingface.co/transformers/pretrained_models.html. 

Please verify that this Jupyter notebook is running the Python Neuron kernel environment that was set up according to the [PyTorch Neuron Installation Guide](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/neuron-intro/pytorch-setup/pytorch-install.html). You can select the kernel from the "Kernel -> Change Kernel" option on the top of this Jupyter notebook page.

## Install Dependencies:
This tutorial requires the following pip packages:

- `torch-neuron`
- `neuron-cc[tensorflow]`
- `transformers`

Most of these packages will be installed when configuring your environment using the Neuron PyTorch setup guide. The additional dependencies must be installed here.

In [ ]:
!pip install --upgrade "transformers==4.6.0"

### Compile the model into an AWS Neuron optimized TorchScript


In [ ]:
import tensorflow  # to work around a protobuf version conflict issue
import torch
import torch.neuron
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import transformers
import os
import warnings

Now setup the number of cores. For an inf1.xlarge or inf1.2xlarge instance, we will set num_cores = 4. For an inf1.6xlarge, set num_cores = 16.

In [ ]:
# Setting up NeuronCore groups for inf1.6xlarge with 16 cores
num_cores = 4
os.environ["TOKENIZERS_PARALLELISM"]="false"

Now, build the tokenizer from the pre-trained model and also create the model.

In [ ]:
# Build tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc", return_dict=False)

It's time to setup some example inputs and also max lengths. This will help in defining the paraphrase and non-paraphrase inputs.

In [ ]:
# Setup some example inputs
sequence_0 = "Hugging Face is a technology company based in New York City"
sequence_1 = "Apples are especially good for your health"
sequence_2 = "Hugging Face's headquarters are situated in Manhattan"

max_length=128
paraphrase = tokenizer.encode_plus(sequence_0, sequence_2, max_length=max_length, padding='max_length', truncation=True, return_tensors="pt")
not_paraphrase = tokenizer.encode_plus(sequence_0, sequence_1, max_length=max_length, padding='max_length', truncation=True, return_tensors="pt")

Now, let's run the encoded example sequences through the original PyTorch model. We will also convert the example sequences to a format that is compatible with TorchScript tracing, which takes place in a subsequent step.

In [ ]:
# Run the original PyTorch model on compilation exaple
paraphrase_classification_logits = model(**paraphrase)[0]
print(f"Model output: {paraphrase_classification_logits}")

# Convert example inputs to a format that is compatible with TorchScript tracing
example_inputs_paraphrase = paraphrase['input_ids'], paraphrase['attention_mask'], paraphrase['token_type_ids']
example_inputs_not_paraphrase = not_paraphrase['input_ids'], not_paraphrase['attention_mask'], not_paraphrase['token_type_ids']

Now, the final step is to trace and save the Neuron-compiled Torchscript model

In [ ]:
# Run torch.neuron.trace to generate a TorchScript that is optimized by AWS Neuron
model_neuron = torch.neuron.trace(model, example_inputs_paraphrase)

# Verify the TorchScript works on both example inputs
paraphrase_classification_logits_neuron = model_neuron(*example_inputs_paraphrase)
not_paraphrase_classification_logits_neuron = model_neuron(*example_inputs_not_paraphrase)

# Save the TorchScript for later use
model_neuron.save('bert_neuron.pt')

You may inspect `model_neuron.graph` to see which part is running on CPU versus running on the accelerator. All native `aten` operators in the graph will be running on CPU.

In [ ]:
print(model_neuron.graph)



### Deploy the AWS Neuron optimized TorchScript

To deploy the AWS Neuron optimized TorchScript, you may choose to load any already-compiled model from disk to avoid the compilation step.

In [ ]:
# Load TorchScript back
model_neuron = torch.jit.load('bert_neuron.pt')

# Verify the TorchScript works on both example inputs
paraphrase_classification_logits_neuron = model_neuron(*example_inputs_paraphrase)
not_paraphrase_classification_logits_neuron = model_neuron(*example_inputs_not_paraphrase)

classes = ['not paraphrase', 'paraphrase']
paraphrase_prediction = paraphrase_classification_logits_neuron[0][0].argmax().item()
not_paraphrase_prediction = not_paraphrase_classification_logits_neuron[0][0].argmax().item()

print('BERT says that "{}" and "{}" are {}'.format(sequence_0, sequence_2, classes[paraphrase_prediction]))
print()
print('BERT says that "{}" and "{}" are {}'.format(sequence_0, sequence_1, classes[not_paraphrase_prediction]))

Now let's run the model in parallel on four cores. For this, define couple of utility functions.

In [ ]:
def get_input_with_padding(batch, batch_size, max_length):
    ## Reformulate the batch into three batch tensors - default batch size batches the outer dimension
    encoded = batch['encoded']
    inputs = torch.squeeze(encoded['input_ids'], 1)
    attention = torch.squeeze(encoded['attention_mask'], 1)
    token_type = torch.squeeze(encoded['token_type_ids'], 1)
    quality = list(map(int, batch['quality']))

    if inputs.size()[0] != batch_size:
        print("Input size = {} - padding".format(inputs.size()))
        remainder = batch_size - inputs.size()[0]
        zeros = torch.zeros( [remainder, max_length], dtype=torch.long )
        inputs = torch.cat( [inputs, zeros] )
        attention = torch.cat( [attention, zeros] )
        token_type = torch.cat( [token_type, zeros] )

    assert(inputs.size()[0] == batch_size and inputs.size()[1] == max_length)
    assert(attention.size()[0] == batch_size and attention.size()[1] == max_length)
    assert(token_type.size()[0] == batch_size and token_type.size()[1] == max_length)

    return (inputs, attention, token_type), quality

def count(output, quality):
    assert output.size(0) >= len(quality)
    correct_count = 0
    count = len(quality)
    
    batch_predictions = [ row.argmax().item() for row in output ]

    for a, b in zip(batch_predictions, quality):
        if int(a)==int(b):
            correct_count += 1

    return correct_count, count

### Data parallel inference
In the following cells, we use the data parallel approach for inference. In this approach, we load multiple models, all of them running in parallel. Each model is loaded onto a single NeuronCore. In the below implementation, we launch 4 models, thereby utilizing all the 4 cores on an inf1.xlarge or inf1.2xlarge instance.

By running more than 1 model concurrently, the model throughput of the system increases. To accurate benchmark the system, we need to efficiently feed the models so as to keep them busy at all times. In the below setup, this is done by using a producer-consumer model. We maintain a common Python queue shared across all the models. The common queue enables feeding data continuously to the models.

First, we download two helper libraries called `parallel` and `bert_benchmark_utils` and a test dataset called `glue_mrpc_dev.tsv`:

In [ ]:
!wget https://raw.githubusercontent.com/aws-neuron/aws-neuron-sdk/master/src/examples/pytorch/bert_tutorial/parallel.py -O parallel.py
!wget https://raw.githubusercontent.com/aws-neuron/aws-neuron-sdk/master/src/examples/pytorch/bert_tutorial/bert_benchmark_utils.py -O bert_benchmark_utils.py
!wget https://raw.githubusercontent.com/aws-neuron/aws-neuron-sdk/master/src/examples/pytorch/bert_tutorial/glue_mrpc_dev.tsv -O glue_mrpc_dev.tsv

Next we proceed with importing the required libraries and setting up the input file and its configurations:

In [ ]:
from parallel import NeuronSimpleDataParallel
from bert_benchmark_utils import BertTestDataset, BertResults
import time
import functools

max_length = 128
num_cores = 4
batch_size = 1

tsv_file="glue_mrpc_dev.tsv" ## where to locate this file on local??

data_set = BertTestDataset( tsv_file=tsv_file, tokenizer=tokenizer, max_length=max_length )
data_loader = torch.utils.data.DataLoader(data_set, batch_size=batch_size, shuffle=True)

Now, let us create a handler function which will aggregate the result. Also, this is where you create the Data Parallel model object using the `NeuronSimpleDataParallel()` wrapper.

In [ ]:
#Result aggregation class (code in bert_benchmark_utils.py)
results = BertResults(batch_size, num_cores)
def result_handler(output, result_id, start, end, input_dict):
    correct_count, inference_count = count(output[0], input_dict.pop(result_id))
    elapsed = end - start
    results.add_result(correct_count, inference_count, [elapsed], [end], [start])

parallel_neuron_model = NeuronSimpleDataParallel('bert_neuron.pt', num_cores)

<br/>

Now, the final step is perform parallel inference and calculate the benchmark:

In [ ]:
#Starting the inference threads
parallel_neuron_model.start_continuous_inference()

# Warm up the cores - the first inference request to a Neuron-compiled model causes the model
# to be loaded onto a NeuronCore. We do not want model loading time to be included in the benchmark.
z = torch.zeros( [batch_size, max_length], dtype=torch.long )
batch = (z, z, z)
for _ in range(num_cores*4):
    parallel_neuron_model.infer(batch, -1, None)
    
input_dict = {}
input_id = 0
for _ in range(50):
    for batch in data_loader:
        batch, quality = get_input_with_padding(batch, batch_size, max_length)
        input_dict[input_id] = quality
        callback_fn = functools.partial(result_handler, input_dict=input_dict)
        parallel_neuron_model.infer(batch, input_id, callback_fn)
        input_id+=1

# Stop inference                
parallel_neuron_model.stop()


with open("benchmark.txt", "w") as f:
    results.report(f, window_size=1)

with open("benchmark.txt", "r") as f:
    for line in f:
        print(line)

<br/>

Note that in the above benchmarking test we are using a batch size of `1` which will provide good latency but will not optimize for throughput

If you feel inclined, you can retry the benchmarking with a larger batch size to understand its impact on latency and throughput. You can do this by:

1. modifying the original notebook code to create an example input with the new batch size
2. re-tracing the original model with the new example input
3. re-running the above benchmark code